### Implement inferences and output monitoring 

Inference :- Simulation-based inference (SBI) is a class of methods that infer the input parameters and unobservable latent variables in a simulator from observational data.

Monitoring :- This section provides information to monitor the system state during simulation in Production System Simulation.

#### Event tracing

In order to debug or visualize a simulation,we want to trace when events are created, triggered and processed.we also want to trace which process created an event and which processes waited for an event.

The two most interesting functions for these use-cases are Environment.step(), where all events get processed, and Environment.schedule(), where all events get scheduled and inserted into SimPy’s event queue.


In [1]:
from functools import partial, wraps
import simpy

In [2]:
def trace(env, callback):
    """
        Replace the ``step()`` method of *env* with a tracing function that 
        calls *callbacks* with an events time, priority, ID and its instance just before it is processed.
    
    """
    def get_wrapper(env_step, callback):
        """
            Generate the wrapper for env.step().
        """
        @wraps(env_step)
        def tracing_step():
            """
                Call *callback* for the next event if one exist before calling ``env.step()``.
            """
            if len(env._queue):
                t, prio, eid, event = env._queue[0]
                callback(t, prio, eid, event)
            return env_step()
        return tracing_step
    env.step = get_wrapper(env.step, callback)

In [3]:
def monitor(data, t, prio, eid, event):
    data.append((t, eid, type(event)))

In [4]:
def test_process(env):
    yield env.timeout(1)

In [5]:
data = [] # Bind *data* as first argument to monitor()
monitor = partial(monitor, data)
env = simpy.Environment()
trace(env, monitor)
p = env.process(test_process(env))
env.run(until=p)

In [6]:
for d in data:
    print(d)

(0, 0, <class 'simpy.events.Initialize'>)
(1, 1, <class 'simpy.events.Timeout'>)
(1, 2, <class 'simpy.events.Process'>)
